In [18]:
from selenium import webdriver      
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup as bs
import pandas as pd

verbose = True

url = 'https://www.imdb.com/title/tt13024974/reviews?spoiler=hide&sort=totalVotes&dir=desc&ratingFilter=0'

load_more_button_xpath = '//*[@id="load-more-trigger"]'  

options = webdriver.ChromeOptions()
options.add_argument('headless')

driver = webdriver.Chrome('./chromedriver.exe', options=options)
driver.get(url)

# Load More 싹다 누르기
while True:
  try:
    load_more_button = driver.find_element(By.XPATH, load_more_button_xpath)
    load_more_button.send_keys(Keys.ENTER)
    time.sleep(1)
  except Exception as e:
    print(e)
    break

soup = bs(driver.page_source, 'html.parser')


text_list = []
rating_list = []
title_list = []
user_list = []
date_list = []
review_df = pd.DataFrame()

try:
  i = 1
  while True:
    main_selector = f'#main > section > div.lister > div.lister-list > div:nth-child({i}) > div.review-container > div.lister-item-content'
    
    text_selector = f'{main_selector} > div.content > div.text.show-more__control'
    text = soup.select(text_selector)[0].text
    
    rating_selector = f'{main_selector} > div.ipl-ratings-bar > span > span:nth-child(2)'
    rating = soup.select(rating_selector)[0].text
          
    title_selector = f'{main_selector} > a'
    title = soup.select(title_selector)[0].text
    
    user_selector = f'{main_selector} > div.display-name-date > span.display-name-link > a'
    user = soup.select(user_selector)[0].text
    
    date_selector = f'{main_selector} > div.display-name-date > span.review-date'
    date = soup.select(date_selector)[0].text
    
    text_list.append(text)
    rating_list.append(rating)
    title_list.append(title)
    user_list.append(user)
    date_list.append(date)
      
    if verbose:
      print(f"{i} is done")
      
    i += 1
except:
  a = 1 # 아무 의미 없음
  
if verbose:
  print('{title_id} end')

review_df['review'] = text_list
review_df['rating'] = rating_list
review_df['title'] = title_list
review_df['user'] = user_list
review_df['date'] = date_list
  


C:\Users\Hi\AppData\Local\Temp\ipykernel_20392\1826738933.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe', options=options)


Message: element not interactable
  (Session info: headless chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0054A813+48355]
	(No symbol) [0x004DC4B1]
	(No symbol) [0x003E5220]
	(No symbol) [0x0040D046]
	(No symbol) [0x0040C8B1]
	(No symbol) [0x0042A73C]
	(No symbol) [0x00409A36]
	(No symbol) [0x0042AA94]
	(No symbol) [0x0043C922]
	(No symbol) [0x0042A536]
	(No symbol) [0x004082DC]
	(No symbol) [0x004093DD]
	GetHandleVerifier [0x007AAABD+2539405]
	GetHandleVerifier [0x007EA78F+2800735]
	GetHandleVerifier [0x007E456C+2775612]
	GetHandleVerifier [0x005D51E0+616112]
	(No symbol) [0x004E5F8C]
	(No symbol) [0x004E2328]
	(No symbol) [0x004E240B]
	(No symbol) [0x004D4FF7]
	BaseThreadInitThunk [0x767500C9+25]
	RtlGetAppContainerNamedObjectPath [0x77EA7B4E+286]
	RtlGetAppContainerNamedObjectPath [0x77EA7B1E+238]

1 is done
2 is done
3 is done
4 is done
5 is done
6 is done
7 is done
8 is done
9 is done
10 is done
11 is done
12 is done
13 is done
14 is done
15 is done
16 is don

In [19]:
review_df

,review,rating,title,user,date
0,This is a stunning example of how cinema or TV...,10,Doesn't pull its punches\n,Jodie_ds,28 August 2021
1,This drama deals with very hard and sensitive ...,10,"Will make you hate human nature, brilliant pe...",hanalisss,28 August 2021
2,Wow! D. P. Is fast moving and gripping - I had...,10,Dark and insightful\n,vicknair-66656,29 August 2021
3,Must watch series from Netflix KOREA.1. The pr...,10,Eye opening. Must watch\n,j_ahgase,29 August 2021
4,Trailer gave me a good impression. Supporting ...,10,Worth it\n,vickspet,27 August 2021
...,...,...,...,...,...
62,Binge watched the entire series. Didn't expect...,9,Superb\n,chaparala-abhilash,15 January 2022
63,This was both one of the very best and most de...,9,Thought Provoking\n,aznidentity,29 December 2021
64,It's an amazing story and wonderful piece of t...,9,Amazing yet horrifying\n,loptr-24929,7 September 2021
65,"The Season 1 is nice story, good nicely made a...",8,Too short drama\n,mailtaskservice,2 June 2022


In [1]:
from selenium import webdriver      
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

def get_review(title_id: str, verbose: bool=False) -> tuple[str, "pd.DataFrame"]:
  
  """IMDb에서 review data를 sraping해오는 func
  + dynamic crawling으로 전체 review 데이터 긁어오기

  Args:
      title_id (str): 영화별 고유 id
      verbose (bool, optional): scraping 과정을 출력할건지 여부

  Returns:
      tuple[str, DataFrame]: title_id, [rating, title, review,	user, date] column을 가진 dataframe
  """
  
  # url = f'https://www.imdb.com/title/{title_id}/reviews?sort=totalVotes&dir=desc&ratingFilter=0'
  # spolier 같은 경우는 접혀있어서 눌러서 봐야함. 따라서 오류가 생긴다. Hide Spoiler옵션 누른 url로 대체
  url = f'https://www.imdb.com/title/{title_id}/reviews?spoiler=hide&sort=totalVotes&dir=desc&ratingFilter=0'

  options = webdriver.ChromeOptions()
  options.add_argument('headless')

  driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
  driver.get(url)
  
  load_more_button_xpath = '//*[@id="load-more-trigger"]'  

  # Load More 싹다 누르기
  while True:
    try:
      load_more_button = driver.find_element(By.XPATH, load_more_button_xpath)
      load_more_button.send_keys(Keys.ENTER)
      time.sleep(1)
    except Exception as e:
      # print(e)
      break

  soup = bs(driver.page_source, 'html.parser')
  
  text_list = []
  rating_list = []
  title_list = []
  user_list = []
  date_list = []
  review_df = pd.DataFrame()
  
  try:
    i = 1
    while True:
      main_selector = f'#main > section > div.lister > div.lister-list > div:nth-child({i}) > div.review-container > div.lister-item-content'
      
      text_selector = f'{main_selector} > div.content > div.text.show-more__control'
      text = soup.select(text_selector)[0].text
      
      rating_selector = f'{main_selector} > div.ipl-ratings-bar > span > span:nth-child(2)'
      rating = soup.select(rating_selector)[0].text
            
      title_selector = f'{main_selector} > a'
      title = soup.select(title_selector)[0].text
      
      user_selector = f'{main_selector} > div.display-name-date > span.display-name-link > a'
      user = soup.select(user_selector)[0].text
      
      date_selector = f'{main_selector} > div.display-name-date > span.review-date'
      date = soup.select(date_selector)[0].text
      
      text_list.append(text)
      rating_list.append(rating)
      title_list.append(title)
      user_list.append(user)
      date_list.append(date)
        
      if verbose:
        print(f"{i} is done")
        
      i += 1
  except:
    a = 1 # 아무 의미 없음
    
  if verbose:
    print('f{title_id} end')
  
  review_df['review'] = text_list
  review_df['rating'] = rating_list
  review_df['title'] = title_list
  review_df['user'] = user_list
  review_df['date'] = date_list
  
  return title_id, review_df

In [4]:
title_id, review_df = get_review('tt13024974', True)

1 is done
2 is done
3 is done
4 is done
5 is done
6 is done
7 is done
8 is done
9 is done
10 is done
11 is done
12 is done
13 is done
14 is done
15 is done
16 is done
17 is done
18 is done
19 is done
20 is done
21 is done
22 is done
23 is done
24 is done
25 is done
26 is done
27 is done
28 is done
29 is done
30 is done
31 is done
32 is done
33 is done
34 is done
35 is done
36 is done
37 is done
38 is done
39 is done
40 is done
41 is done
42 is done
43 is done
44 is done
45 is done
46 is done
47 is done
48 is done
49 is done
50 is done
51 is done
52 is done
53 is done
54 is done
55 is done
56 is done
57 is done
58 is done
59 is done
60 is done
61 is done
62 is done
63 is done
64 is done
65 is done
66 is done
67 is done
{title_id} end


In [3]:
title_id, review_df

('tt13024974',
                                                review rating  \
 0   This is a stunning example of how cinema or TV...     10   
 1   This drama deals with very hard and sensitive ...     10   
 2   Wow! D. P. Is fast moving and gripping - I had...     10   
 3   Must watch series from Netflix KOREA.1. The pr...     10   
 4   Trailer gave me a good impression. Supporting ...     10   
 ..                                                ...    ...   
 62  Binge watched the entire series. Didn't expect...      9   
 63  This was both one of the very best and most de...      9   
 64  It's an amazing story and wonderful piece of t...      9   
 65  The Season 1 is nice story, good nicely made a...      8   
 66  This Netflix D. P. K-Drama focusing on compuls...     10   
 
                                                 title                user  \
 0                          Doesn't pull its punches\n            Jodie_ds   
 1    Will make you hate human nature, brillian